In [ ]:
import kwant
from math import sin, cos
import matplotlib.pyplot as plt
import numpy as np

import scipy.constants
import cmath

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
from sns_system import get_templates

In [ ]:
def NOT(shape):
    """returns not(shape)"""
    return lambda site: not shape(site)

def UNION(*shapes):
    """returns shape which is an OR function applied to shapes"""
    return lambda site: any(shape(site) for shape in shapes)
    
def INTERSECTION(*shapes):
    """returns shape which is an AND function applied to shapes"""
    return lambda site: all(shape(site) for shape in shapes)
    
def DIFFERENCE(shape_A, shape_B):
    """returns shape which is true when the site is in the first shape, but not in the second"""
    return lambda site: shape_A(site) and not shape_B(site)

def EDGE(shape): 
    def _shape(site):
        sites = [('wsite', [-1, 0]), ('esite', [1, 0]),
                 ('nsite', [0, 1]), ('ssite', [0, -1]),
                 ('nwsite', [-1, 1]), ('nesite', [1, 1]),
                 ('swsite', [-1, -1]), ('sesite', [1, -1])]
        s = lambda x: kwant.builder.Site(site.family, site.tag + x)
        neighboring_sites = {k: s(x) for k, x in sites}
        return shape(site) and not all(map(shape, neighboring_sites.values()))
    return _shape

def INTERIOR(shape):
    return DIFFERENCE(shape, EDGE(shape))

def TRANSLATE(shape, vector):
    def _shape(site):
        translated_site  = kwant.builder.Site(site.family, site.tag - vector)
        return shape(translated_site)
    return _shape


def below_curve(curve):
    def _shape(site):
        x, y = site.pos
        return y < curve(x)
    return _shape

def above_curve(curve):
    return NOT(below_curve(curve))

def left_of_curve(curve):
    def _shape(site):
        x, y = site.pos
        return x < curve(y)
    return _shape

def right_of_curve(curve):
    return NOT(left_of_curve(curve))

In [ ]:
def add_to_site_colors(site_colors, marked_sites, color):
    site_colors.update({site: color for site in marked_sites})
    
def site_color_function(site_color, syst):
    return [site_color[site] for site in syst.sites]


# Make simple builder

In [ ]:
import scipy.interpolate
def create_parallel_sine(distance, z_x, z_y):
    def _parallel_sine(x, distance, z_x, z_y):
        g       = lambda t: z_y * sin(2*np.pi/z_x*t)
        g_prime = lambda t: z_y * 2*np.pi/z_x*cos(2*np.pi/z_x*t)
        def _x(t):
            return t - distance*g_prime(t)/np.sqrt(1 + g_prime(t)**2) - x

        def y(t):
            return g(t) + distance/np.sqrt(1 + g_prime(t)**2)

        t = fsolve(_x, x)
        return y(t)

    xdim = np.linspace(0, z_x, 1000)
    ydim = [_parallel_sine(x, distance, z_x, z_y) for x in xdim]
    
    parallel_sine = scipy.interpolate.interp1d(xdim, ydim)
    
#     return lambda x: scipy.interpolate.interp1d(xdim, ydim)(x%z_x)
    return lambda x: parallel_sine(x%z_x)

In [ ]:
L_m = 150
L_m = 200

L_x = 2400
L_sc_up = 200
L_sc_down = 200

z_x = 1200

z_y = 125
z_y = 125

a   = 10

parallel_curve = False
parallel_curve = True


transverse_soi = True
mu_from_bottom_of_spin_orbit_bands = True
k_x_in_sc = True

In [ ]:
def make_system():
    template_barrier, template_normal, template_sc_left, template_sc_right = get_templates(
            a, transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)

    template_interior = template_normal
    template_edge = template_barrier
    template_top_superconductor = template_sc_left
    template_bottom_superconductor = template_sc_right

    #----------------------
    # Define middle section
    curve = lambda x: z_y*sin(2*np.pi / z_x * x)
    below_shape = below_curve(lambda x: curve(x) + L_m//2)
    above_shape = above_curve(lambda x: curve(x) - L_m//2)

    if parallel_curve:
        curve = create_parallel_sine(0, z_x, z_y)

        curve_top = create_parallel_sine(L_m//2, z_x, z_y)
        below_shape = below_curve(curve_top)

        curve_bottom = create_parallel_sine(-L_m//2, z_x, z_y)
        above_shape = above_curve(curve_bottom)

    right_boundary = left_of_curve(lambda y: L_x)
    left_boundary = right_of_curve(lambda y: 0)
    middle_shape = INTERSECTION(below_shape, above_shape, right_boundary, left_boundary)


    #------------
    # Define edge
    edge_shape = EDGE(middle_shape)
    edge_initial_site = (0,0)

    #------------------------
    # Remove edge from middle
    interior_shape = INTERIOR(middle_shape)
    interior_initial_site = (a, a)

    #--------------------------
    # Define top superconductor
    top_superconductor_border = lambda x: L_sc_up + L_m//2 + z_y
    top_superconductor_shape = INTERSECTION(DIFFERENCE(below_curve(top_superconductor_border), middle_shape),
                                            left_boundary, right_boundary)
    top_superconductor_initial_site = (0,L_m)

    #-----------------------------
    # Define bottom superconductor
    bottom_superconductor_border = lambda x: -L_sc_down - L_m//2 - z_y
    bottom_superconductor_shape = INTERSECTION(DIFFERENCE(above_curve(bottom_superconductor_border), middle_shape),
                                            left_boundary, right_boundary)
    bottom_superconductor_initial_site = (0, -L_m-a)

    #-----------
    # Define cut
    cut_curve_top = lambda x: curve(x) + a
    cut_curve_bottom = lambda x: cut_curve_top(x) - a

    top_cut_shape = EDGE(below_curve(cut_curve_top))
    bottom_cut_shape = EDGE(below_curve(cut_curve_bottom))


    ##################
    ## Build system ##
    ##################
    
    site_colors = dict()

    #---------------
    # Create builder
    if wraparound:
        syst = kwant.Builder(kwant.TranslationalSymmetry([0, z_x]))
    else:
        syst = kwant.Builder()

    #----------
    # Fill edge
    edge_sites = syst.fill(template_edge, edge_shape, edge_initial_site)
    add_to_site_colors(site_colors, edge_sites, 'red')
    
    #--------------
    # Fill interior
    interior_sites = syst.fill(template_interior, interior_shape, interior_initial_site)
    add_to_site_colors(site_colors, interior_sites, 'grey')

    #------------------------
    # Fill top superconductor
    top_superconductor_sites = syst.fill(template_top_superconductor, top_superconductor_shape, top_superconductor_initial_site)
    add_to_site_colors(site_colors, top_superconductor_sites, 'gold')

    #---------------------------
    # Fill bottom superconductor
    bottom_superconductor_sites = syst.fill(template_bottom_superconductor, bottom_superconductor_shape, bottom_superconductor_initial_site)
    add_to_site_colors(site_colors, bottom_superconductor_sites, 'gold')

    
    ##########################################
    ## Add features for current calculation ##
    ##########################################

    #---------------------
    # Make cut for current
    top_cut = list(site for site in syst.sites() if top_cut_shape(site))
    add_to_site_colors(site_colors, top_cut, 'teal')
    bottom_cut = list(site for site in syst.sites() if bottom_cut_shape(site))
    add_to_site_colors(site_colors, bottom_cut, 'blue')

    
    syst = supercurrent_matsubara.add_vlead(syst, norbs, *cuts)
    
    #####################
    ## Finalize system ##
    #####################
    
    if wraparound:
        kwant.wraparound.wraparound(syst)
    
    syst = syst.finalized()
    return syst, site_colors, hoppings

In [ ]:
kwant.plot(syst, site_color=site_color_function(site_colors, syst), dpi=200);

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = .3,
                 phase = np.pi,
                 T = 0.0,
                 V = 0)

params = dict(**constants,
              **params_raw)

In [ ]:
import spectrum

In [ ]:
density = kwant.operator.Density(syst)

In [ ]:
e, ev = spectrum.calc_spectrum(syst, params, k=4)

In [ ]:
ax=plt.axes()
kwant.plot(syst, site_color=site_color_function(site_colors, syst), dpi=200, ax=ax);
kwant.plotter.map(syst, density(ev[:,0]), dpi=200, ax=ax);
e

In [ ]:
def f(B):
    _params = params.copy()
    _params['B'] = B
    return spectrum.calc_lowest_state((None, _params), syst=syst)

In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
learner = adaptive.Learner1D(f, [0, 2])


In [ ]:
runner = adaptive.Runner(learner, ntasks=24)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
learner.plot()

In [ ]:
learner.plot()

In [ ]:
learner.plot()